In [ ]:
from pyomo.environ import *
import math

In [ ]:
N = 1

D1_elements = {1, 2}
D2_elements = {}

v0_data = {}
v0_data[1] = 10
v0_data[2] = 20

s0_data = {}
s0_data[1] = 140
s0_data[2] = 151.5

trange = range(0, 4*N+1)

l = 4
b = 3
w = 6
alpha = 5
beta = 10
vmax = 30

a_data = [alpha, 0, -beta, 0] * N

In [ ]:
model = ConcreteModel()

In [ ]:
# Sets
model.D1 = Set(ordered=False, initialize=D1_elements)
model.D2 = Set(ordered=False, initialize=D2_elements)
model.C = Set(ordered=False, initialize=model.D1 | model.D2)

In [ ]:
# Parameters
model.v0 = Param(model.C, initialize=v0_data)
model.s0 = Param(model.C, initialize=s0_data)
model.a = Param(trange, initialize={i+1: a_data[i] for i in range(len(a_data))})

In [ ]:
# Variables
model.t = Var(model.C, range(1, 4*N+1), within=NonNegativeReals)

In [ ]:
# Defining critical velocities (also enforcing speed limit)
model.v = Var(model.C, trange, within=NonNegativeReals, bounds=(0, vmax))
def v_def_rule(model, i, j):
    if j == 0:
        return model.v[i, j] == model.v0[i]
    else:
        return model.v[i, j] == model.v[i, j-1] + model.a[j] * model.t[i, j]
model.v_definition = Constraint(model.C, trange, rule = v_def_rule)

In [ ]:
# Defining critical distances
model.s = Var(model.C, trange, within=Reals)
def s_def_rule(model, i, j):
    if j == 0:
        return model.s[i, j] == model.s0[i]
    else:
        return model.s[i, j] == model.s[i, j-1] - (model.v[i, j-1] + model.v[i, j]) * model.t[i, j] / 2
model.s_definition = Constraint(model.C, trange, rule = s_def_rule)

In [ ]:
# Defining critical times
model.T = Var(model.C, trange, within=NonNegativeReals)
def T_def_rule(model, i, j):
    if j == 0:
        return model.T[i, j] == 0
    else:
        return model.T[i, j] == model.T[i, j-1] + model.t[i, j]
model.T_definition = Constraint(model.C, trange, rule = T_def_rule)

In [ ]:
# Enforcing car exit at ti4N
def car_exit_rule(model, i):
    return model.s[i, 4*N] == 0
model.car_exit_constraint = Constraint(model.C, rule=car_exit_rule)

In [ ]:
# Preventing in-lane collision
def inlane_collision_rule(model):
    for i in model.C:
        for j in model.C:
            for m in trange:
                for n in trange:
                    if i == j:
                        return Constraint.Skip

                    M1 = 100000
                    M2 = 100000
                    M3 = 100000

                    y1 = Var(domain=Binary)
                    y2 = Var(domain=Binary)
                    y3 = Var(domain=Binary)

                    yield model.T[i, m] - model.T[j, n-1] <= M1*(1-y1)
                    yield model.T[j, n] - model.T[i, m] <= M2*(1-y2)
                    deltaT = model.T[i, m] - model.T[j, n-1]
                    yield model.s[j, n-1] - (model.v[j, n-1]*deltaT + model.a[n]*deltaT*deltaT/2) - model.s[i, m] + l*math.copysign(1, model.s0[i] - model.s0[j]) <= M3*(1-y3)
                    yield y1+y2+y3 >= 1

model.inlane_collision_constraint = ConstraintList(rule=inlane_collision_rule)

In [ ]:
# Objective
def objective_rule(model):
    return sum(model.T[i, 4*N] for i in model.C)
model.objective = Objective(rule=objective_rule, sense=minimize)

In [ ]:
model.display()

In [ ]:
model.pprint()

In [ ]:
opt = SolverFactory("gurobi_direct")
results = opt.solve(model)

In [ ]:
for i in model.C:
    for j in range(1, 4*N+1):
        print(f"t{i, j} = {model.t[i, j].value}")
        print(f"s{i, j} = {model.s[i, j].value}")
        print(f"v{i, j} = {model.v[i, j].value}")
        print()